In [1]:
!date

Thu Feb  4 15:59:32 PST 2021


# Supplementary Figure 1

In [2]:
import glob
import pandas as pd
import numpy as np
import pandas as pd
import scipy as scp
import sklearn
import itertools
from scipy.optimize import fsolve
from upsetplot import generate_data, plot, from_memberships
from collections import Counter
from matplotlib.ticker import FormatStrFormatter
from matplotlib.ticker import StrMethodFormatter
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.patches as mpatches
import matplotlib.ticker as ticker

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 22})
%config InlineBackend.figure_format = 'retina'

In [3]:
v2_names = np.array(['SRR8599150_v2',
       'heart1k_v2', 'SRR8611943_v2',
       'SRR8257100_v2', 'EMTAB7320_v2',
       'SRR7299563_v2', 'SRR8513910_v2',
       'SRR8639063_v2', 'SRR8524760_v2',
       'SRR6956073_v2', 'hgmm1k_v2',
       'SRR8206317_v2', 'SRR8327928_v2',
       'SRR6998058_v2'], dtype=object)

v3_names = np.array(['pbmc_1k_v3', 'hgmm10k_v3',
       'neuron_10k_v3', 'pbmc_10k_v3',
       'heart1k_v3', 'hgmm1k_v3'], dtype=object)

In [4]:
v2_data = {}
v2_data["EMTAB7320_v2"]  = {"n_reads": 335147976}
v2_data["heart1k_v2"]  = {"n_reads": 88872840}
v2_data["hgmm1k_v2"] = {"n_reads": 75225120}
v2_data["SRR6956073_v2"] = {"n_reads": 161274652}
v2_data["SRR6998058_v2"] = {"n_reads": 37227612}
v2_data["SRR7299563_v2"]  = {"n_reads": 112176350}
v2_data["SRR8206317_v2"] = {"n_reads": 85992089}
v2_data["SRR8257100_v2"]  = {"n_reads": 189337914}
v2_data["SRR8327928_v2"] = {"n_reads": 190094560}
v2_data["SRR8513910_v2"]  = {"n_reads": 146617182}
v2_data["SRR8524760_v2"]  = {"n_reads": 97106426}
v2_data["SRR8599150_v2"]  = {"n_reads": 8860361}
v2_data["SRR8611943_v2"]  = {"n_reads": 21574502}
v2_data["SRR8639063_v2"]  = {"n_reads": 416437344}

v2_data["EMTAB7320_v2"]["n_cells"] = 4510
v2_data["heart1k_v2"]["n_cells"] = 712
v2_data["hgmm1k_v2"]["n_cells"] = 1079
v2_data["SRR6956073_v2"]["n_cells"] = 4168
v2_data["SRR6998058_v2"]["n_cells"] = 575
v2_data["SRR7299563_v2"]["n_cells"] = 1660
v2_data["SRR8206317_v2"]["n_cells"] = 4418
v2_data["SRR8257100_v2"]["n_cells"] = 11685
v2_data["SRR8327928_v2"]["n_cells"] = 10396
v2_data["SRR8513910_v2"]["n_cells"] = 726
v2_data["SRR8524760_v2"]["n_cells"] = 3064
v2_data["SRR8599150_v2"]["n_cells"] = 3949
v2_data["SRR8611943_v2"]["n_cells"] = 5194
v2_data["SRR8639063_v2"]["n_cells"] = 6614

In [5]:
v3_data = {}
v3_data["hgmm1k_v3"] = {"n_reads": 63105786}
v3_data["neuron_10k_v3"] = {"n_reads": 357111595}
v3_data["pbmc_10k_v3"] = {"n_reads": 638901019}
v3_data["pbmc_1k_v3"] = {"n_reads": 66601887}
v3_data["heart1k_v3"] = {"n_reads": 84512390}
v3_data["hgmm10k_v3"] = {"n_reads": 721180737}

v3_data["hgmm1k_v3"]["n_cells"] = 1011
v3_data["neuron_10k_v3"]["n_cells"] = 11477
v3_data["pbmc_10k_v3"]["n_cells"] = 1045
v3_data["pbmc_1k_v3"]["n_cells"] = 11790
v3_data["heart1k_v3"]["n_cells"] = 11692
v3_data["hgmm10k_v3"]["n_cells"] = 1227

In [6]:
%%time
for f in glob.glob("/home/sina/projects/bus/validate/all_bus/k*"):
    name = "_".join(f.split("/")[7].split("_")[1:])
    print(name)
    if "v2" in name:
        # Estimate number of UMIs
        v2_data[name]["Nest"] = pd.read_csv(f + "/Nest.txt", header=None).values.flatten()
        
    elif "v3" in name:
        v3_data[name]["Nest"] = pd.read_csv(f + "/Nest.txt", header=None).values.flatten()

CPU times: user 592 µs, sys: 231 µs, total: 823 µs
Wall time: 8.14 ms


# Supplementary Figure 1

In [7]:
data = [np.array(v2_data[i]["Nest"]) for i in v2_names]
[data.append(np.array(v3_data[i]["Nest"])) for i in v3_names]

names = v2_names 
n3 = v3_names
names = np.append(names, n3)

nreads = [v2_data[i]["n_reads"] for i in v2_names]
[nreads.append(v3_data[i]["n_reads"]) for i in v3_names]

idx_sorted = np.argsort(nreads)
sorted_nreads = np.sort(nreads)

KeyError: 'Nest'

In [ ]:
data = list(np.array(data)[idx_sorted])

names = names[idx_sorted]

In [ ]:
kallisto_color_v2 = '#cccccc'
kallisto_color_v3 = 'black'
tick_spacing = 1


def adjacent_values(vals, q1, q3):
    upper_adjacent_value = q3 + (q3 - q1) * 1.5
    upper_adjacent_value = np.clip(upper_adjacent_value, q3, vals[-1])

    lower_adjacent_value = q1 - (q3 - q1) * 1.5
    lower_adjacent_value = np.clip(lower_adjacent_value, vals[0], q1)
    return lower_adjacent_value, upper_adjacent_value


def set_axis_style(ax, labels):
    ax.get_xaxis().set_tick_params(direction='out')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_xticks(np.arange(1, len(labels) + 1))
    ax.set_xticklabels(labels)
    ax.set_xlim(0.25, len(labels) + 0.75)
    #ax.set_xlabel('Sample name')

fig, ax = plt.subplots(figsize=(20, 10), nrows=1, ncols=1)


parts = ax.violinplot(data, showmeans=False, showmedians=False,showextrema=False)


for pc_i in range(len(parts['bodies'])):
    if "v2" in names[pc_i]:
        parts['bodies'][pc_i].set_facecolor(kallisto_color_v2)
    elif "v3" in names[pc_i]:
        parts['bodies'][pc_i].set_facecolor(kallisto_color_v3)
    parts['bodies'][pc_i].set_edgecolor('black')
    parts['bodies'][pc_i].set_alpha(1)

# set style for the axes
set_axis_style(ax, names)

#ticks = ax.get_yticks().tolist()
#ticks = ["{:.1f}".format(i) for i in ticks]
#ticks[0] = "No counts lost 0.0"
#ticks[-1] = "All counts lost 1.0"

#for tick in ax.get_xticklabels():
#    tick.set_rotation(45, ha="right")

ax.set_xticklabels(ax.xaxis.get_majorticklabels(), rotation=45, ha="right")

grey_patch = mpatches.Patch(color=kallisto_color_v2, label='v2 Chemistry')
black_patch = mpatches.Patch(color=kallisto_color_v3, label='v3 Chemistry')

ax.figure.legend(handles=[grey_patch, black_patch], loc=1, bbox_to_anchor=(0.25,0.93))

ax.set_ylabel("Estimated number of UMIs per bead")

ax.set_yscale('log')

ax.grid(True, which="minor")
ax.set_ylim(bottom=10000)

plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
plt.subplots_adjust(bottom=0.15, wspace=0.05)
plt.tight_layout()
plt.savefig("sizeU.pdf")
plt.show()